การควบคุมเหมาะที่สุดและการเรียนรูู้เสริมกำลัง -- ดร.วโรดม ตู้จินดา

# ภาคผนวก C รวมการใช้งานเครื่องมือ Drake

Drake [1] เป็นเครื่องมือซอฟต์แวร์สำหรับหุ่นยนต์ที่พัฒนาโดย Russ Tedrake และทีมงาน 
สนับสนุนโดย Toyota Research Institute (TRI) ในช่วงเริ่มต้น API ใช้ภาษา C++ เป็นหลัก 
แต่ในปัจจุบันรองรับภาษาไพทอนโดยผ่านแพ็กเกจ pydrake ซึ่งโค้ดในหนังสือนี้มีการใช้งานในหลายส่วน 
เช่นการหาอนุพันธ์อัตโนมัติ และการจำลองโมเดล

อย่างไรก็ตามความช่วยเหลือเมื่อเกิดปัญหาอาจค่อนข้างจำกัด 
เนื่องจากชุมชนของ Drake ยังมีขนาดเล็ก ดังนั้นในภาคผนวกนี้จึงรวมการใช้งานแพ็กเกจ pydrake 
รวมถึงการปรับแต่งแก้ไขเพื่อให้ใช้งานได้กับโค้ดในแต่ละบทของหนังสือ 

## C.1 การหาอนุพันธ์อัตโนมัติ

ในการควบคุมเหมาะที่สุด พลวัตของโมเดลมักอยู่ในรูปสมการอนุพันธ์ ดังนั้นมักต้องการการคำนวณหาอนุพันธ์ในโปรแกรม 
ซึ่งสามารถใช้วิธีการดังต่อไปนี้

**1. วิธีหาคำตอบเชิงสัญลักษณ์ (symbolic method)**

ใช้แพ็กเกจเสริมที่สามารถหาอนุพันธ์จากกฎหรือสูตรเหมือนกับที่เราใช้หาคำตอบบนกระดาษ

* ข้อดี
    * มีความแม่นยำสูงเทียบเท่าทางทฤษฎี
    * เข้าใจได้ง่าย

* ข้อเสีย
    * ต้องการแพ็กเกจที่สนับสนุนการคำนวณเชิงสัญลักษณ์
    * เมื่อพจน์มีความซับซ้อนมาก การคำนวณจะซับซ้อนตาม ทำให้ใช้เวลานานในการหาคำตอบ และผลลัพธ์อาจยังติดในรูปที่สามารถทำให้ง่ายกว่าโดยมนุษย์

**2. วิธีการประมาณค่าเชิงเลข (numerical method)** 

ใช้การประมาณค่า เช่นผลต่างจำกัด 
(finite difference) $f'(x) \approx \underset{h \rightarrow 0}{lim}\frac{f(x+h)-f(x)}{h}$  
(หรือใช็ $f'(x) \approx \underset{h \rightarrow 0}{lim}\frac{f(x+0.5h)-f(x-0.5h)}{h}$ จะได้ความแม่นยำมากขึ้น)

* ข้อดี
    * เขียนโค้ดง่าย ไม่ต้องการแพ็กเกจเสริมสำหรับคำนวณอนุพันธ์
    * ทำงานเร็ว

* ข้อเสีย
    * เป็นเพียงการประมาณค่าดังนั้นจึงมีค่าผิดพลาด โดยความแม่นยำขึ้นกับขนาดของขั้น $h$
    * เมื่อเลีอก $h$ มีค่าน้อยมาก จะเกิดปัญหาเชิงเลข

**3. วิธีการหาอนุพันธ์อัตโนมัติ (automatic differentiation)**

เป็นวิธีการหาอนุพันธ์โดยใช้ลำดับของการคำนวณขั้นพื้นฐานเช่นการบวกลบคูณหาร และฟังก์ชันเช่นเลขชี้กำลัง (exponential) ตรีโกณ ลอการิทึม ผ่านใช้กฎลูกโซ่ (chain rule)

* ข้อดี
    * ใช้การคำนวณน้อย ทำให้ได้คำตอบเร็ว
    * มีความแม่นยำสูงเทียบเท่าผลลัพธ์ทางทฤษฎี

* ข้อเสีย
    * ต้องการแพ็กเกจเสริมเช่น Drake, JAX, tensorflow ฯลฯ หรือหากเขียนโปรแกรมเองจะต้องมีการจัดเก็บค่าของฟังก์ชันและอนุพันธ์ไปพร้อมกัน

โดยข้อดีของวิธีการ 3 ทำให้เป็นที่นิยมในการหาอนุพันธ์สำหรับระบบที่มีความซับซ้อนเช่นโครงข่ายประสาทเทียม การหาอนุพันธ์อัตโนมัติสามารถแบ่งย่อยเป็น 2 ประเภทหลักคือ

* การคำนวณไปข้างหน้า (forward) 
* การคำนวณผันกลับ (reverse)

การเลือกใช้แบบใดจึงเหมาะสมขึ้นกับจำนวนอินพุตและเอาต์พุตของฟังก์ชัน เช่นหากมีอินพุตจำนวนมากและเอาต์พุตเป็นสเกลาร์หรือจำนวนน้อย (เช่นโครงข่ายประสาทเทียม) 
ควรเลือกวิธีการคำนวณผันกลับ แต่สำหรับพลวัตในหนังสือนี้มิใช่ประเด็นสำคัญ 
ดังนั้นจะเลือกใช้วิธีการคำนวณไปข้างหน้า ซึ่ง Drake รองรับผ่านแพ็กเกจย่อย forwarddiff


In [1]:
from pydrake import forwarddiff

**ตัวอย่าง C.1** ทดสอบกับฟังก์ชันพื้นฐาน $y = f_1(x) = xsin(x)$

In [5]:
import numpy as np
def f1(x):
    return x*np.sin(x)

อนุพันธ์มีค่าเท่ากับ $f_1'(x) = xcos(x) + sin(x)$ เมื่อกำหนดให้ $x = \frac{\pi}{6}$  
เอาต์พุตของฟังก์ชันและอนุพันธ์มีค่าเป็นดังนี้

In [11]:
x1 = np.pi/6
y = f1(x1)
print("y = {}".format(y))

y = 0.26179938779914935


In [13]:
dy = forwarddiff.derivative(f1,x1)
print("dy = {}".format(dy))

dy = 0.9534498410585543


เมื่อเปรียบเทียบกับการคำนวณด้วยมือ จะพบว่าความแตกต่างเป็นศูนย์

In [16]:
dy_a = x1*np.cos(x1) + np.sin(x1)
print("dy (คำนวณด้วยมือ) = {}".format(dy_a))
print("ค่าแตกต่าง = {}".format(np.linalg.norm(dy - dy_a)))

dy (คำนวณด้วยมือ) = 0.9534498410585543
ค่าแตกต่าง = 0.0


เปรียบเทียบกับวิธีประมาณค่าโดยผลต่างจำกัดโดยใช้ขั้นเวลาน้อยเช่น $h = 10^{-6}$ จะพบว่ามีความแม่นยำน้อยกว่า 
(ซึ่งหากใช้ค่า $h$ น้อยเกินไป จะเกิดปัญหาเชิงเลขทำให้ค่าผิดพลาดเพิ่มขึ้น)

In [23]:
h = 1e-6
dy_fd = (f1(x1+0.5*h) - f1(x1-0.5*h))/h
print("dy (ประมาณค่าเชิงเลข) = {}".format(dy_fd))
print("ค่าแตกต่าง = {}".format(np.linalg.norm(dy_fd - dy_a)))

dy (ประมาณค่าเชิงเลข) = 0.9534498409680126
ค่าแตกต่าง = 9.054179628265047e-11


**ตัวอย่าง C.2** กรณีที่ฟังก์ชันมีอินพุตเป็นเวกเตอร์และเอาต์พุตเป็นสเกลาร์เช่น $f_2(x) = x_1^2 + 2x_2^2$ 

In [24]:
def f2(x):
    return x[0]**2 + 2*x[1]**2

อนุพันธ์จะเป็นเวกเตอร์
$$
f_2'(x) = 
\left[\begin{array}{c} 
\frac{\partial f_2(x)}{\partial x_1} \\
\frac{\partial f_2(x)}{\partial x_2} 
\end{array}\right] = 
\left[\begin{array}{c} 
2x_1 \\
4x_2
\end{array}\right] \tag{C.1}
$$

เรียกว่าเกรเดียนต์ จะใช้ forwarddiff.gradient สมมุติว่ากำหนด $x_2 = [1.0, -1.0]$ ได้อนุพันธ์เป็นดังนี้ (ลองเปรียบเทียบกับการคำนวณด้วยมือจะเห็นว่าเท่ากัน)

In [26]:
x2 = np.array([1.0,-1.0])
grad_fx2 = forwarddiff.gradient(f2,x2)
print(grad_fx2)

[ 2. -4.]


**ตัวอย่าง C.3** ระบบพลวัตที่เป็นเงื่อนไขบังคับของการควบคุมเหมาะที่สุดเช่น $x_{k+1} = f(x_k,u_k)$ 
หรือในกรณีทั่วไปคือ $y = f(x,u)$ อยู่ในรูปหลายอินพุตหลายเอาต์พุต เพื่อความง่ายพิจารณากรณีฟังก์ชันไม่เป็นเชิงเส้นที่ไม่มีอินพุต $u$

$$
\left[\begin{array}{c}
y_1 \\
y_2
\end{array}\right] =
\left[\begin{array}{c}
x_1^3  + 2x_2 \\
x_1cos(x_2)
\end{array}\right] \tag{C.2}
$$

อนุพันธ์อยู่ในรูปเมทริกซ์เรียกว่าจาโคเบียน
$$
\left[\begin{array}{cc}
\frac{\partial y_1}{\partial x_1} & \frac{\partial y_1}{\partial x_2} \\
\frac{\partial y_2}{\partial x_1} & \frac{\partial y_2}{\partial x_2} \\
\end{array}\right] = 
\left[\begin{array}{cc}
3x_1^2 & 2 \\
cos(x_2) & -x_1sin(x_2) \\
\end{array}\right] \tag{C.3}
$$
 

In [32]:
def f3(x):
    y0 = x[0]**3  + 2*x[1]
    y1 = x[0]*np.cos(x[1])
    return [y0,y1]

คำนวณอนุพันธ์ที่อินพุต $x = [1, \pi /2]$ โดยใช้ forwarddiff.jacobian 

In [33]:
x3 = np.array([1.0, np.pi/2])
y3 = forwarddiff.jacobian(f3,x3)
print(y3)

[[ 3.000000e+00  2.000000e+00]
 [ 6.123234e-17 -1.000000e+00]]


### C.1.1 แก้ไขโค้ดจาก forwarddiff.py

โด้ดจากแพ็กเกจ forwarddiff ของ Drake เขียนไว้สำหรับปัญหาพื้นฐาน ซึ่งหลายครั้งต้องการการปรับแต่งเพื่อให้ได้ผลตามต้องการ 

## บรรณานุกรม

1. R. Tedrake and the Drake Development Team. ["Drake: Model-based design and verification
 for robotics"](https://drake.mit.edu). 2019.

<div align="center">
<img src="https://raw.githubusercontent.com/dewdotninja/sharing-github/refs/heads/master/dewninja_logo50.jpg" alt="dewninja"/>
</div>
<div align="center">dew.ninja 2025</div>